In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
import zipfile
import os

# Specify the path to the zip file
zip_file_path = '/content/drive/MyDrive/paper1/level3_diffusion.zip'  # Replace with the actual path to your zip file

# Specify the directory where you want to extract the contents
extracted_folder_path = '/content/drive/MyDrive/paper1/'  # Replace with the desired extraction path

# Create the target extraction folder if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# List the contents of the extraction folder
extracted_contents = os.listdir(extracted_folder_path)
print(f"Contents of the extraction folder: {extracted_contents}")


Contents of the extraction folder: ['final_folder.zip', 'Level2.zip', 'level3_diffusion.zip', 'final_folder', '__MACOSX', 'Level2', 'level3_diffusion']


In [9]:
train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/paper1/level3_diffusion', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

In [10]:
resnet18 = models.resnet18(pretrained=True)

# Modify the last fully connected layer for 8-class classification
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 8)

# Transfer the model to GPU if available
resnet18 = resnet18.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

In [11]:
num_epochs = 4
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

print("Training finished")

Epoch 1/4, Loss: 1.9416473171927713
Epoch 2/4, Loss: 1.482788044756109
Epoch 3/4, Loss: 1.2097174471074885
Epoch 4/4, Loss: 1.0588125976649199
Training finished


In [13]:
torch.save(resnet18.state_dict(), '/content/drive/MyDrive/paper1/fine_tuned_resnet18_level3_diffusion.pth')

In [14]:
import torch
from torchvision import transforms
from PIL import Image

# Define the transformation for inference
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the fine-tuned model
model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 8)  # Assuming 2 classes
model.load_state_dict(torch.load('/content/drive/MyDrive/paper1/fine_tuned_resnet18_level3_diffusion.pth'))
model.eval()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)



In [15]:
import zipfile
import os

# Specify the path to the zip file
zip_file_path = '/content/drive/MyDrive/paper1/level3_diffusion_test.zip'  # Replace with the actual path to your zip file

# Specify the directory where you want to extract the contents
extracted_folder_path = '/content/drive/MyDrive/paper1/'  # Replace with the desired extraction path

# Create the target extraction folder if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

# List the contents of the extraction folder
extracted_contents = os.listdir(extracted_folder_path)
print(f"Contents of the extraction folder: {extracted_contents}")

Contents of the extraction folder: ['final_folder.zip', 'Level2.zip', 'level3_diffusion.zip', 'final_folder', '__MACOSX', 'Level2', 'level3_diffusion', 'fine_tuned_resnet18_level1_dalle+real.pth', 'level3_gans.zip', 'level3_gans_1.zip', 'level3_gans_1', 'level3_gans', 'fine_tuned_resnet18_level2.pth', 'fine_tuned_resnet18_level3_diffusion.pth', 'fine_tuned_resnet18_level3_gans.pth', 'level3_gans_test.zip', 'level3_diffusion_test.zip', 'level3_diffusion_test']


In [19]:
# Load and preprocess the image for inference
image_path = '/content/drive/MyDrive/paper1/level3_diffusion/idm_200/aakydlwfer_idm200.png'
image = Image.open(image_path).convert('RGB')
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0).to(device)  # Add batch dimension and move to device

# Perform inference
with torch.no_grad():
    output = model(input_batch)

# Get predicted class
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()

# Print results
print(f"Predicted class: {predicted_class}")
print(f"Class probabilities: {probabilities}")


Predicted class: 7
Class probabilities: tensor([4.9796e-03, 4.0944e-04, 7.2526e-04, 2.2582e-04, 2.2546e-02, 2.8021e-01,
        5.3701e-02, 6.3721e-01])


In [20]:
import os
from PIL import Image

# Specify the path to the folder containing the image files
folder_path = '/content/drive/MyDrive/paper1/level3_diffusion_test/'

# List all files in the folder
l = ["dalle","glide_100_10","glide_100_27","glide_50_27","guided","idm_100","idm_100_cfg","idm_200"]
count3 = 0
for i in l:
  files = os.listdir(folder_path+i)

# Filter out only image files (you may need to adjust the extensions based on your images)
  image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
  count = 1
  count2 = 0
# Iterate over image files and open each image
  for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path+i, image_file)
    # img_emb = model.encode(Image.open(image_path))
    # Open the image using Pillow
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
          output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()

        # Print results
        print(f"Predicted class: {predicted_class}")
        print(f"Class probabilities: {probabilities}")

        if(predicted_class == count3):
          print("yes")
          count2 += 1
        image.close()
        print(count)
        count += 1
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
  print(f"the percentage of predicting {i} images is : {(count2*100)/count}")
  count3 += 1

Predicted class: 1
Class probabilities: tensor([0.0860, 0.3457, 0.2155, 0.1595, 0.0642, 0.0216, 0.0640, 0.0436])
1
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/dalle/yqqmbglayb_dalle.png
Predicted class: 0
Class probabilities: tensor([9.7471e-01, 2.6536e-04, 5.3680e-04, 2.0334e-04, 8.7496e-04, 3.0773e-03,
        8.7691e-03, 1.1562e-02])
yes
2
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/dalle/xqlumlknls_dalle.png
Predicted class: 0
Class probabilities: tensor([8.8621e-01, 2.2946e-04, 1.5193e-04, 1.6225e-04, 6.8393e-03, 1.7960e-02,
        6.9849e-02, 1.8600e-02])
yes
3
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/dalle/ybbbdeihcf_dalle.png
Predicted class: 0
Class probabilities: tensor([6.9747e-01, 1.7794e-05, 8.3906e-05, 1.5660e-05, 1.0268e-02, 1.0212e-01,
        1.0596e-01, 8.4072e-02])
yes
4
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/dalle/yhxjifarjw_dalle.png
Predicted class: 0
Class probabiliti

In [21]:
import os
from PIL import Image

# Specify the path to the folder containing the image files
folder_path = '/content/drive/MyDrive/paper1/level3_diffusion_test/'

# List all files in the folder
l = ["dalle","glide_100_10","glide_100_27","glide_50_27","guided","idm_100","idm_100_cfg","idm_200"]
count3 = 0
for i in l[1:4]:
  files = os.listdir(folder_path+i)

# Filter out only image files (you may need to adjust the extensions based on your images)
  image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
  count = 1
  count2 = 0
# Iterate over image files and open each image
  for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path+i, image_file)
    # img_emb = model.encode(Image.open(image_path))
    # Open the image using Pillow
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
          output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()

        # Print results
        print(f"Predicted class: {predicted_class}")
        print(f"Class probabilities: {probabilities}")

        if(predicted_class >= 1 or predicted_class <= 3):
          print("yes")
          count2 += 1
        image.close()
        print(count)
        count += 1
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
  print(f"the percentage of predicting {i} images is : {(count2*100)/count}")
  count3 += 1

Predicted class: 2
Class probabilities: tensor([0.0115, 0.0990, 0.2870, 0.1179, 0.0681, 0.1115, 0.0498, 0.2552])
yes
1
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/glide_100_10/alzrdyourf 1.57.12 PM.png
Predicted class: 1
Class probabilities: tensor([3.2214e-04, 3.6135e-01, 2.8404e-01, 3.3591e-01, 1.6539e-02, 1.0271e-03,
        3.2181e-04, 4.8844e-04])
yes
2
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/glide_100_10/ayzndlluru 1.57.12 PM.png
Predicted class: 1
Class probabilities: tensor([0.0082, 0.3364, 0.3254, 0.2898, 0.0127, 0.0057, 0.0143, 0.0076])
yes
3
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/glide_100_10/ccatehwpeu 1.57.12 PM.png
Predicted class: 2
Class probabilities: tensor([0.0120, 0.2174, 0.3871, 0.2431, 0.0169, 0.0287, 0.0237, 0.0712])
yes
4
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/glide_100_10/cmjapngcpn 1.57.12 PM.png
Predicted class: 3
Class probabilities: tensor([9.1592e-04, 3.36

In [22]:
import os
from PIL import Image

# Specify the path to the folder containing the image files
folder_path = '/content/drive/MyDrive/paper1/level3_diffusion_test/'

# List all files in the folder
l = ["dalle","glide_100_10","glide_100_27","glide_50_27","guided","idm_100","idm_100_cfg","idm_200"]
count3 = 0
for i in l[5:]:
  files = os.listdir(folder_path+i)

# Filter out only image files (you may need to adjust the extensions based on your images)
  image_files = [file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
  count = 1
  count2 = 0
# Iterate over image files and open each image
  for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path+i, image_file)
    # img_emb = model.encode(Image.open(image_path))
    # Open the image using Pillow
    try:
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image)
        input_batch = input_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
          output = model(input_batch)

        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()

        # Print results
        print(f"Predicted class: {predicted_class}")
        print(f"Class probabilities: {probabilities}")

        if(predicted_class >= 5):
          print("yes")
          count2 += 1
        image.close()
        print(count)
        count += 1
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Error opening image {image_path}: {e}")
  print(f"the percentage of predicting {i} images is : {(count2*100)/count}")
  count3 += 1

Predicted class: 0
Class probabilities: tensor([0.3214, 0.0697, 0.0682, 0.0636, 0.0537, 0.1466, 0.1575, 0.1193])
1
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/idm_100/afwwethmvf 1.55.32 PM.png
Predicted class: 7
Class probabilities: tensor([0.0019, 0.0006, 0.0011, 0.0010, 0.0265, 0.4218, 0.0399, 0.5072])
yes
2
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/idm_100/bjipeebzgu 1.55.32 PM.png
Predicted class: 2
Class probabilities: tensor([0.0032, 0.2107, 0.2573, 0.1256, 0.0821, 0.0727, 0.0170, 0.2314])
3
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/idm_100/ajldduomvc 1.55.32 PM.png
Predicted class: 5
Class probabilities: tensor([0.0023, 0.0006, 0.0020, 0.0005, 0.1468, 0.4622, 0.0644, 0.3211])
yes
4
Opened image: /content/drive/MyDrive/paper1/level3_diffusion_test/idm_100/cdjghtyvsn 1.55.33 PM.png
Predicted class: 7
Class probabilities: tensor([1.3017e-04, 2.6842e-04, 2.1340e-03, 1.7112e-04, 3.8918e-04, 3.9371e-01,
        4.93

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
